In [1]:
import rasterio
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.util import view_as_blocks
import tifffile

In [2]:
img_folder = '/lithology_paper/lithology_events/Raster' 
save_folder = '/lithology_paper/lithology_events/Patches'
patch_size = 128

In [3]:
def open_raster(path):
    '''
    Open a raster file given it's path
    Inputs: path(string) - full path of the image
    Ouput: image(numpy array) - the image in numpy array format
    '''
    raster = rasterio.open(path)
    band_ls = []
  
    #Open every band 
    b=1
    while b<=raster.count:
        band = raster.read(b)
        band_ls.append(band)
        b = b+1
      
    #Create numpy array
    image = np.stack(band_ls)
    raster.close()
    return image

def make_patches(img, patch_size):
    '''Create patches of a raster file given a patch size.
    Inputs: img(3D numpy array) - the image to be divided
            patch_size(int) - the patch size
    Ouputs: patches(4D numpy array) - the created patches with shape
                                      (number of patches, bands, patch size, patch size)
    '''
    bands = img.shape[0]
    patches = view_as_blocks(img, (bands, patch_size, patch_size))
    patches = patches.reshape(-1, bands, patch_size, patch_size)
    return patches

In [ ]:
i = 1
print('Creating patches...')
for img in os.listdir(img_folder):
    if 'EMSR122' in img:
        continue
    print('{}/{}:{}'.format(i, (len(os.listdir(img_folder)) - 1), img))
    img_path = os.path.join(img_folder, img)
    img_np = open_raster(img_path)
    patches = make_patches(img_np, patch_size)
    num_patches = patches.shape[0]
    i = i + 1
    
    for patch in range(num_patches):
        name_base = img.split('.')[0]
        save_name = name_base + '_patch{}.tif'.format(patch + 1)
        save_path = os.path.join(save_folder, save_name)
        tifffile.imsave(save_path, patches[patch])